<a href="https://colab.research.google.com/github/JGisaina/Boss-workshops/blob/main/Molecular_dynamics_tutorial_GROMACS_BioExcel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Protein Ligand Complex MD using the BioExcel Building Blocks library (biobb)

This tutorial aims to illustrate the process of setting up a simulation system containing a protein in complex with a ligand, step by step, using the BioExcel Building Blocks library (biobb). The particular example used is the T4 lysozyme L99A/M102Q protein (PDB code 3HTB, https://doi.org/10.2210/pdb3HTB/pdb), in complex with the 2-propylphenol small molecule (3-letter Code JZ4, https://www.rcsb.org/ligand/JZ4).

## Setup Installation

In [1]:
# Only executed when using google colab
import sys
if 'google.colab' in sys.modules:
  import subprocess
  from pathlib import Path
  try:
    subprocess.run(["conda", "-V"], check=True)
  except FileNotFoundError:
    subprocess.run([sys.executable, "-m", "pip", "install", "condacolab"], check=True)
    import condacolab
    condacolab.install()
    # Clone repository
    repo_URL = "https://github.com/bioexcel/biobb_wf_protein-complex_md_setup.git"
    repo_name = Path(repo_URL).name.split('.')[0]
    if not Path(repo_name).exists():
      subprocess.run(["mamba", "install", "-y", "git"], check=True)
      subprocess.run(["git", "clone", repo_URL], check=True)
      print("⏬ Repository properly cloned.")
    # Install environment
    print("⏳ Creating environment...")
    env_file_path = f"{repo_name}/conda_env/environment.yml"
    subprocess.run(["mamba", "env", "update", "-n", "base", "-f", env_file_path], check=True)
    print("🎨 Install NGLView dependencies...")
    subprocess.run(["mamba", "install", "-y", "-c", "conda-forge", "nglview==3.0.8", "ipywidgets=7.7.2"], check=True)
    print("👍 Conda environment successfully created and updated.")

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:18
🔁 Restarting kernel...
⏬ Repository properly cloned.
⏳ Creating environment...
🎨 Install NGLView dependencies...
👍 Conda environment successfully created and updated.


In [1]:
# Enable widgets for colab
if 'google.colab' in sys.modules:
  from google.colab import output
  output.enable_custom_widget_manager()
  import os
  os.chdir("biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks")
  print(f"📂 New working directory: {os.getcwd()}")

📂 New working directory: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks


In [3]:
!jupyter --version

Selected Jupyter core packages...
IPython          : 8.24.0
ipykernel        : 6.29.3
ipywidgets       : 7.7.2
jupyter_client   : 8.6.1
jupyter_core     : 5.7.2
jupyter_server   : 2.14.0
jupyterlab       : 4.2.0
nbclient         : 0.10.0
nbconvert        : 7.16.4
nbformat         : 5.10.4
notebook         : 7.2.0
qtconsole        : 5.5.2
traitlets        : 5.14.3


In [2]:
from google.colab import output
output.enable_custom_widget_manager()

In [4]:
!jupyter-nbextension enable nglview --py --sys-prefix

Traceback (most recent call last):
  File "/usr/local/bin/jupyter-nbextension", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/jupyter_core/application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/notebook/nbextensions.py", line 972, in start
    super().start()
  File "/usr/local/lib/python3.10/dist-packages/jupyter_core/application.py", line 270, in start
    self.subapp.start()
  File "/usr/local/lib/python3.10/dist-packages/notebook/nbextensions.py", line 882, in start
    self.toggle_nbextension_python(self.extra_args[0])
  File "/usr/local/lib/python3.10/dist-packages/notebook/nbextensions.py", line 855, in toggle_nbextension_python
    return toggle(module,
  File "/usr/local/lib/python3.10/dist-packages/notebook/nbextensions.py

Support for third party widgets will remain active for the duration of the session. To disable support:

In [5]:
import nglview
import ipywidgets
import os
import zipfile

from biobb_io.api.pdb import pdb
from biobb_structure_utils.utils.extract_heteroatoms import extract_heteroatoms
from biobb_structure_utils.utils.extract_molecule import extract_molecule
from biobb_structure_utils.utils.cat_pdb import cat_pdb
from biobb_model.model.fix_side_chain import fix_side_chain
from biobb_gromacs.gromacs.pdb2gmx import pdb2gmx
from biobb_chemistry.ambertools.reduce_add_hydrogens import reduce_add_hydrogens
from biobb_chemistry.babelm.babel_minimize import babel_minimize
from biobb_chemistry.acpype.acpype_params_gmx import acpype_params_gmx
from biobb_gromacs.gromacs.make_ndx import make_ndx
from biobb_gromacs.gromacs.genrestr import genrestr
from biobb_analysis.gromacs.gmx_trjconv_str import gmx_trjconv_str
from biobb_structure_utils.utils.cat_pdb import cat_pdb
from biobb_gromacs.gromacs_extra.append_ligand import append_ligand
from biobb_gromacs.gromacs.editconf import editconf
from biobb_gromacs.gromacs.solvate import solvate
from biobb_gromacs.gromacs.grompp import grompp
from biobb_gromacs.gromacs.genion import genion
from biobb_gromacs.gromacs.mdrun import mdrun
from biobb_analysis.gromacs.gmx_energy import gmx_energy
from biobb_gromacs.gromacs.make_ndx import make_ndx
from biobb_analysis.gromacs.gmx_rms import gmx_rms
from biobb_analysis.gromacs.gmx_rgyr import gmx_rgyr
from biobb_analysis.gromacs.gmx_image import gmx_image
from biobb_analysis.gromacs.gmx_trjconv_str import gmx_trjconv_str

import plotly
import plotly.graph_objs as go

Pipeline steps:

    Input Parameters
    Fetching PDB Structure
    Fix Protein Structure
    Create Protein System Topology
    Create ligand system topology
    Preparing Ligand Restraints
    Create new protein-ligand complex structure file
    Create new protein-ligand complex topology file
    Create Solvent Box
    Fill the Box with Water Molecules
    Adding Ions
    Energetically Minimize the System
    Equilibrate the System (NVT)
    Equilibrate the System (NPT)
    Free Molecular Dynamics Simulation
    Post-processing and Visualizing Resulting 3D Trajectory
    Output Files

## Input parameters

Input parameters needed:

1. **pdbCode**: PDB code of the protein-ligand complex structure (e.g. 3HTB, https://doi.org/10.2210/pdb3HTB/pdb)
2. **ligandCode**: Small molecule 3-letter code for the ligand structure (e.g. JZ4, https://www.rcsb.org/ligand/JZ4)
3. **mol_charge**: Charge of the small molecule, needed to add hydrogen atoms.


In [6]:

pdbCode = "3HTB"
ligandCode = "JZ4"
mol_charge = 0

## Fetching PDB structure

Downloading PDB structure with the protein-ligand complex from the RCSB PDB database.
Alternatively, a PDB file can be used as starting structure.
Splitting the molecule in three different files:

1. proteinFile: Protein structure
2. ligandFile: Ligand structure
3. complexFile: Protein-ligand complex structure


Building Blocks used:

    Pdb from biobb_io.api.pdb


In [7]:
# Downloading desired PDB file
# Import module
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
downloaded_pdb = pdbCode+'.orig.pdb'
prop = {
    'pdb_code': pdbCode,
    'filter': False
}

# Create and launch bb
pdb(output_pdb_path=downloaded_pdb,
    properties=prop)

2024-05-21 13:23:44,901 [MainThread  ] [INFO ]  Executing biobb_io.api.pdb Version: 4.1.0
2024-05-21 13:23:44,905 [MainThread  ] [INFO ]  Downloading 3htb from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb3htb.ent
2024-05-21 13:23:45,889 [MainThread  ] [INFO ]  Writting pdb to: 3HTB.orig.pdb


0

In [8]:
# Extracting Protein, Ligand and Protein-Ligand Complex to three different files
# Import module
#from biobb_structure_utils.utils.extract_heteroatoms import extract_heteroatoms
#from biobb_structure_utils.utils.extract_molecule import extract_molecule
#from biobb_structure_utils.utils.cat_pdb import cat_pdb

# Create properties dict and inputs/outputs
proteinFile = pdbCode+'.pdb'
ligandFile = ligandCode+'.pdb'
complexFile = pdbCode+'_'+ligandCode+'.pdb'

prop = {
     'heteroatoms' : [{"name": "JZ4"}]
}

extract_heteroatoms(input_structure_path=downloaded_pdb,
     output_heteroatom_path=ligandFile,
     properties=prop)

extract_molecule(input_structure_path=downloaded_pdb,
     output_molecule_path=proteinFile)

print(proteinFile, ligandFile, complexFile)

cat_pdb(input_structure1=proteinFile,
       input_structure2=ligandFile,
       output_structure_path=complexFile)

2024-05-21 13:23:49,306 [MainThread  ] [INFO ]  Executing biobb_structure_utils.utils.extract_heteroatoms Version: 4.1.0
2024-05-21 13:23:49,311 [MainThread  ] [INFO ]  Copy: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/3HTB.orig.pdb to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/4f701a0b-cf57-4b11-88e9-a3c07648ad0d
2024-05-21 13:23:49,346 [MainThread  ] [INFO ]  Writting pdb to: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/4f701a0b-cf57-4b11-88e9-a3c07648ad0d/JZ4.pdb
2024-05-21 13:23:49,349 [MainThread  ] [INFO ]  Removed: ['/content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/4f701a0b-cf57-4b11-88e9-a3c07648ad0d']
2024-05-21 13:23:49,354 [MainThread  ] [INFO ]  Executing biobb_structure_utils.utils.extract_molecule Version: 4.1.0
2024-05-21 13:23:49,357 [MainThread  ] [INFO ]  Copy: /content/biobb_wf_protein-complex_md_setup/

0


###Visualizing 3D structures

Visualizing the generated PDB structures using NGL:

* Protein structure (Left)
* Ligand structure (Center)
* Protein-ligand complex (Right)



In [9]:
# Show structures: protein, ligand and protein-ligand complex
view1 = nglview.show_structure_file(proteinFile)
view1._remote_call('setSize', target='Widget', args=['350px','400px'])
view1.camera='orthographic'
view1
view2 = nglview.show_structure_file(ligandFile)
view2.add_representation(repr_type='ball+stick')
view2._remote_call('setSize', target='Widget', args=['350px','400px'])
view2.camera='orthographic'
view2
view3 = nglview.show_structure_file(complexFile)
view3.add_representation(repr_type='licorice', radius='.5', selection=ligandCode)
view3._remote_call('setSize', target='Widget', args=['350px','400px'])
view3.camera='orthographic'
view3
ipywidgets.HBox([view1, view2, view3])

###Fix protein structure

Checking and fixing (if needed) the protein structure:

* Modeling missing side-chain atoms, modifying incorrect amide assignments, choosing alternative locations.
* Checking for missing backbone atoms, heteroatoms, modified residues and possible atomic clashes.


Building Blocks used:

    FixSideChain from biobb_model.model.fix_side_chain


In [10]:
# Check & Fix Protein Structure
# Import module
#from biobb_model.model.fix_side_chain import fix_side_chain

# Create prop dict and inputs/outputs
fixed_pdb = pdbCode+'_fixed.pdb'

# Create and launch bb
fix_side_chain(input_pdb_path=proteinFile,
             output_pdb_path=fixed_pdb)

2024-05-21 13:26:23,449 [MainThread  ] [INFO ]  Executing biobb_model.model.fix_side_chain Version: 4.1.0
2024-05-21 13:26:23,453 [MainThread  ] [INFO ]  Copy: 3HTB.pdb to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/9d7dde4c-fd3a-427a-a529-ab1482c5edd9
2024-05-21 13:26:23,456 [MainThread  ] [INFO ]  check_structure -i /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/9d7dde4c-fd3a-427a-a529-ab1482c5edd9/3HTB.pdb -o /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/9d7dde4c-fd3a-427a-a529-ab1482c5edd9/3HTB_fixed.pdb --force_save fixside --fix ALL

2024-05-21 13:26:23,938 [MainThread  ] [INFO ]  Exit code 0

2024-05-21 13:26:23,940 [MainThread  ] [INFO ]  ================================================================================
=                   BioBB structure checking utility v3.13.4                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi

0

##Create protein system topology

Building GROMACS topology corresponding to the protein structure.
Force field used in this tutorial is amber99sb-ildn: AMBER parm99 force field with corrections on backbone (sb) and side-chain torsion potentials (ildn). Water molecules type used in this tutorial is spc/e.
Adding hydrogen atoms if missing. Automatically identifying disulfide bridges.

Generating two output files:

* GROMACS structure (gro file)
* GROMACS topology ZIP compressed file containing:
        * GROMACS topology top file (top file)
        * GROMACS position restraint file/s (itp file/s)


Building Blocks used:

    Pdb2gmx from biobb_gromacs.gromacs.pdb2gmx


In [11]:
# Create Protein system topology
# Import module
#from biobb_gromacs.gromacs.pdb2gmx import pdb2gmx

# Create inputs/outputs
output_pdb2gmx_gro = pdbCode+'_pdb2gmx.gro'
output_pdb2gmx_top_zip = pdbCode+'_pdb2gmx_top.zip'
prop = {
    'force_field' : 'amber99sb-ildn',
    'water_type': 'spce'
}

# Create and launch bb
pdb2gmx(input_pdb_path=fixed_pdb,
        output_gro_path=output_pdb2gmx_gro,
        output_top_zip_path=output_pdb2gmx_top_zip,
        properties=prop)

2024-05-21 13:26:43,244 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.pdb2gmx Version: 4.1.0
2024-05-21 13:26:43,247 [MainThread  ] [INFO ]  Copy: 3HTB_fixed.pdb to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/b4933997-84f5-49e6-a28c-22141ffdcadd
2024-05-21 13:26:43,250 [MainThread  ] [INFO ]  GROMACS Pdb2gmx 20222 version detected
2024-05-21 13:26:43,252 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright pdb2gmx -f /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/b4933997-84f5-49e6-a28c-22141ffdcadd/3HTB_fixed.pdb -o /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/b4933997-84f5-49e6-a28c-22141ffdcadd/3HTB_pdb2gmx.gro -p p2g.top -water spce -ff amber99sb-ildn -i posre.itp

2024-05-21 13:26:43,725 [MainThread  ] [INFO ]  Exit code 0

2024-05-21 13:26:43,727 [MainThread  ] [INFO ]  Using the Amber99sb-ildn force field in directory amber99sb-ildn.ff

going to re

0

##Create ligand system topology

Building GROMACS topology corresponding to the ligand structure.
Force field used in this tutorial step is amberGAFF: General AMBER Force Field, designed for rational drug design.

    Step 1: Add hydrogen atoms if missing.
    Step 2: Energetically minimize the system with the new hydrogen atoms.
    Step 3: Generate ligand topology (parameters).


Building Blocks used:

    ReduceAddHydrogens from biobb_chemistry.ambertools.reduce_add_hydrogens
    BabelMinimize from biobb_chemistry.babelm.babel_minimize
    AcpypeParamsGMX from biobb_chemistry.acpype.acpype_params_gmx


###Step 1: Add hydrogen atoms

In [12]:
# Create Ligand system topology, STEP 1
# Reduce_add_hydrogens: add Hydrogen atoms to a small molecule (using Reduce tool from Ambertools package)
# Import module
#from biobb_chemistry.ambertools.reduce_add_hydrogens import reduce_add_hydrogens

# Create prop dict and inputs/outputs
output_reduce_h = ligandCode+'.reduce.H.pdb'
prop = {
    'nuclear' : 'true'
}

# Create and launch bb
reduce_add_hydrogens(input_path=ligandFile,
                   output_path=output_reduce_h,
                   properties=prop)

2024-05-21 13:26:54,169 [MainThread  ] [INFO ]  Executing biobb_chemistry.ambertools.reduce_add_hydrogens Version: 4.1.0
2024-05-21 13:26:54,172 [MainThread  ] [INFO ]  Copy: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/JZ4.pdb to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/9bb0eb38-b037-4675-994a-537f9293058a
2024-05-21 13:26:54,175 [MainThread  ] [INFO ]  reduce -NUClear -OH -ROTNH3 -ALLALT /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/9bb0eb38-b037-4675-994a-537f9293058a/JZ4.pdb > /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/9bb0eb38-b037-4675-994a-537f9293058a/JZ4.reduce.H.pdb

2024-05-21 13:26:54,886 [MainThread  ] [INFO ]  Exit code 0

2024-05-21 13:26:54,887 [MainThread  ] [INFO ]  Processing file: "/content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/9bb0eb38-b037-4675-994a-537f9

0

###Step 2: Energetically minimize the system with the new hydrogen atoms.

In [13]:
# Create Ligand system topology, STEP 2
# Babel_minimize: Structure energy minimization of a small molecule after being modified adding hydrogen atoms
# Import module
#from biobb_chemistry.babelm.babel_minimize import babel_minimize

# Create prop dict and inputs/outputs
output_babel_min = ligandCode+'.H.min.mol2'
prop = {
    'method' : 'sd',
    'criteria' : '1e-10',
    'force_field' : 'GAFF'
}


# Create and launch bb
babel_minimize(input_path=output_reduce_h,
              output_path=output_babel_min,
              properties=prop)

2024-05-21 13:27:01,066 [MainThread  ] [INFO ]  Executing biobb_chemistry.babelm.babel_minimize Version: 4.1.0
2024-05-21 13:27:01,068 [MainThread  ] [INFO ]  Copy: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/JZ4.reduce.H.pdb to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/4be2888c-3321-4d8f-8755-f871c1b0e8c2
2024-05-21 13:27:01,070 [MainThread  ] [INFO ]  Hydrogens  is not correct, assigned default value: False
2024-05-21 13:27:01,072 [MainThread  ] [INFO ]  Steps  is not correct, assigned default value: 2500
2024-05-21 13:27:01,073 [MainThread  ] [INFO ]  Cut-off  is not correct, assigned default value: False
2024-05-21 13:27:01,074 [MainThread  ] [INFO ]  Rvdw  is not correct, assigned default value: 6.0
2024-05-21 13:27:01,075 [MainThread  ] [INFO ]  Rele  is not correct, assigned default value: 10.0
2024-05-21 13:27:01,076 [MainThread  ] [INFO ]  Frequency  is not correct, assigned default value: 

0


###Visualizing 3D structures

Visualizing the small molecule generated PDB structures using NGL:

    Original Ligand Structure (Left)
    Ligand Structure with hydrogen atoms added (with Reduce program) (Center)
    Ligand Structure with hydrogen atoms added (with Reduce program), energy minimized (with Open Babel) (Right)



In [14]:
# Show different structures generated (for comparison)

view1 = nglview.show_structure_file(ligandFile)
view1.add_representation(repr_type='ball+stick')
view1._remote_call('setSize', target='Widget', args=['350px','400px'])
view1.camera='orthographic'
view1
view2 = nglview.show_structure_file(output_reduce_h)
view2.add_representation(repr_type='ball+stick')
view2._remote_call('setSize', target='Widget', args=['350px','400px'])
view2.camera='orthographic'
view2
view3 = nglview.show_structure_file(output_babel_min)
view3.add_representation(repr_type='ball+stick')
view3._remote_call('setSize', target='Widget', args=['350px','400px'])
view3.camera='orthographic'
view3
ipywidgets.HBox([view1, view2, view3])

 ### Step 3: Generate ligand topology (parameters).

In [15]:
# Create Ligand system topology, STEP 3
# Acpype_params_gmx: Generation of topologies for GROMACS with ACPype
# Import module
#from biobb_chemistry.acpype.acpype_params_gmx import acpype_params_gmx

# Create prop dict and inputs/outputs
output_acpype_gro = ligandCode+'params.gro'
output_acpype_itp = ligandCode+'params.itp'
output_acpype_top = ligandCode+'params.top'
output_acpype = ligandCode+'params'
prop = {
    'basename' : output_acpype,
    'charge' : mol_charge
}

# Create and launch bb
acpype_params_gmx(input_path=output_babel_min,
                output_path_gro=output_acpype_gro,
                output_path_itp=output_acpype_itp,
                output_path_top=output_acpype_top,
                properties=prop)

2024-05-21 13:28:24,053 [MainThread  ] [INFO ]  Executing biobb_chemistry.acpype.acpype_params_gmx Version: 4.1.0
2024-05-21 13:28:24,059 [MainThread  ] [INFO ]  Copy: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/JZ4.H.min.mol2 to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/0503ad46-0deb-4adb-a457-4038ff121c73
2024-05-21 13:28:24,062 [MainThread  ] [INFO ]  acpype -i /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/0503ad46-0deb-4adb-a457-4038ff121c73/JZ4.H.min.mol2 -b JZ4params.LR1tpz -n 0

2024-05-21 13:28:27,750 [MainThread  ] [INFO ]  Exit code 0

2024-05-21 13:28:27,752 [MainThread  ] [INFO ]  ==========================================================================
| ACPYPE: AnteChamber PYthon Parser interfacE v. 2022.6.6 (c) 2024 AWSdS |
==> ... charge set to 0
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
==>

0

###Preparing Ligand Restraints



In subsequent steps of the pipeline, such as the equilibration stages of the protein-ligand complex system, it is recommended to apply some restraints to the small molecule, to avoid a possible change in position due to protein repulsion. Position restraints will be applied to the ligand, using a force constant of 1000 kJ/mol*nm^2 on the three coordinates: x, y and z. In this steps the restriction files will be created and integrated in the ligand topology.

    Step 1: Creating an index file with a new group including just the small molecule heavy atoms.
    Step 2: Generating the position restraints file.


Building Blocks used:

    MakeNdx from biobb_gromacs.gromacs.make_ndx
    Genrestr from biobb_gromacs.gromacs.genrestr


###Step 1: Creating an index file for the small molecule heavy atoms

In [16]:
# MakeNdx: Creating index file with a new group (small molecule heavy atoms)
#from biobb_gromacs.gromacs.make_ndx import make_ndx

# Create prop dict and inputs/outputs
output_ligand_ndx = ligandCode+'_index.ndx'
prop = {
    'selection': "0 & ! a H*"
}

# Create and launch bb
make_ndx(input_structure_path=output_acpype_gro,
        output_ndx_path=output_ligand_ndx,
        properties=prop)

2024-05-21 13:28:35,355 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.make_ndx Version: 4.1.0
2024-05-21 13:28:35,358 [MainThread  ] [INFO ]  Copy: JZ4params.gro to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/3f0621aa-ba15-4696-a56f-64c82f2171ff
2024-05-21 13:28:35,361 [MainThread  ] [INFO ]  Copy: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/9e5709ef-475a-487d-ab22-a01ca56a14e9.stdin to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/3f0621aa-ba15-4696-a56f-64c82f2171ff
2024-05-21 13:28:35,363 [MainThread  ] [INFO ]  GROMACS MakeNdx 20222 version detected
2024-05-21 13:28:35,365 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright make_ndx -f /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/3f0621aa-ba15-4696-a56f-64c82f2171ff/JZ4params.gro -o /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setu

0

###Step 2: Generating the position restraints file

In [17]:
# Genrestr: Generating the position restraints file
#from biobb_gromacs.gromacs.genrestr import genrestr

# Create prop dict and inputs/outputs
output_restraints_top = ligandCode+'_posres.itp'
prop = {
    'force_constants': "1000 1000 1000",
    'restrained_group': "System"
}

# Create and launch bb
genrestr(input_structure_path=output_acpype_gro,
         input_ndx_path=output_ligand_ndx,
         output_itp_path=output_restraints_top,
         properties=prop)

2024-05-21 13:28:41,998 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.genrestr Version: 4.1.0
2024-05-21 13:28:42,000 [MainThread  ] [INFO ]  Copy: JZ4params.gro to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/64b5514a-71f6-4bc6-9c66-79c955cda87c
2024-05-21 13:28:42,003 [MainThread  ] [INFO ]  Copy: JZ4_index.ndx to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/64b5514a-71f6-4bc6-9c66-79c955cda87c
2024-05-21 13:28:42,005 [MainThread  ] [INFO ]  Copy: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/212b2f55-692e-4260-8ba2-8318fad9a743.stdin to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/64b5514a-71f6-4bc6-9c66-79c955cda87c
2024-05-21 13:28:42,007 [MainThread  ] [INFO ]  GROMACS Genrestr 20222 version detected
2024-05-21 13:28:42,008 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright genrestr -f /content/biobb_wf_pr

0

##Create new protein-ligand complex structure file

Building new protein-ligand complex PDB file with:

    The new protein system with fixed problems from Fix Protein Structure step and hydrogens atoms added from Create Protein System Topology step.
    The new ligand system with hydrogens atoms added from Create Ligand System Topology step.

This new structure is needed for GROMACS as it is force field-compliant, it has all the new hydrogen atoms, and the atom names are matching the newly generated protein and ligand topologies.

Building Blocks used:

    GMXTrjConvStr from biobb_analysis.gromacs.gmx_trjconv_str


In [18]:
# biobb analysis module
#from biobb_analysis.gromacs.gmx_trjconv_str import gmx_trjconv_str
#from biobb_structure_utils.utils.cat_pdb import cat_pdb

# Convert gro (with hydrogens) to pdb (PROTEIN)
proteinFile_H = pdbCode+'_'+ligandCode+'_complex_H.pdb'
prop = {
    'selection' : 'System'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path=output_pdb2gmx_gro,
              input_top_path=output_pdb2gmx_gro,
              output_str_path=proteinFile_H,
              properties=prop)

# Convert gro (with hydrogens) to pdb (LIGAND)
ligandFile_H = ligandCode+'_complex_H.pdb'
prop = {
    'selection' : 'System'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path=output_acpype_gro,
              input_top_path=output_acpype_gro,
              output_str_path=ligandFile_H,
              properties=prop)


# Concatenating both PDB files: Protein + Ligand
complexFile_H = pdbCode+'_'+ligandCode+'_H.pdb'

# Create and launch bb
cat_pdb(input_structure1=proteinFile_H,
       input_structure2=ligandFile_H,
       output_structure_path=complexFile_H)



2024-05-21 13:28:54,284 [MainThread  ] [INFO ]  Executing biobb_analysis.gromacs.gmx_trjconv_str Version: 4.1.0
2024-05-21 13:28:54,288 [MainThread  ] [INFO ]  Copy: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/3HTB_pdb2gmx.gro to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/28aed783-9f88-4d28-96f6-c39476869759
2024-05-21 13:28:54,292 [MainThread  ] [INFO ]  Copy: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/3HTB_pdb2gmx.gro to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/28aed783-9f88-4d28-96f6-c39476869759
2024-05-21 13:28:54,297 [MainThread  ] [INFO ]  Copy: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/6576a965-88cb-4e2d-86e3-e8a4518f7704.stdin to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/28aed783-9f88-4d28-96f6-c39476869759
2024-05

0

###Create new protein-ligand complex topology file

Building new protein-ligand complex GROMACS topology file with:

    The new protein system topology generated from Create Protein System Topology step.
    The new ligand system topology generated from Create Ligand System Topology step.

NOTE: From this point on, the protein-ligand complex structure and topology generated can be used in a regular MD setup.

Building Blocks used:

    AppendLigand from biobb_gromacs.gromacs_extra.append_ligand (NOTE: link should be updated with the documentation)


In [19]:
# AppendLigand: Append a ligand to a GROMACS topology
# Import module
#from biobb_gromacs.gromacs_extra.append_ligand import append_ligand

# Create prop dict and inputs/outputs
output_complex_top = pdbCode+'_'+ligandCode+'_complex.top.zip'

posresifdef = "POSRES_"+ligandCode.upper()
prop = {
    'posres_name': posresifdef
}

# Create and launch bb
append_ligand(input_top_zip_path=output_pdb2gmx_top_zip,
             input_posres_itp_path=output_restraints_top,
             input_itp_path=output_acpype_itp,
             output_top_zip_path=output_complex_top,
             properties=prop)

2024-05-21 13:29:04,621 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs_extra.append_ligand Version: 4.1.0
2024-05-21 13:29:04,631 [MainThread  ] [INFO ]  Extracting: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/3HTB_pdb2gmx_top.zip
2024-05-21 13:29:04,633 [MainThread  ] [INFO ]  to:
2024-05-21 13:29:04,637 [MainThread  ] [INFO ]  ['2643cddf-ab09-4db9-a1e7-408a9c991886/p2g.top', '2643cddf-ab09-4db9-a1e7-408a9c991886/posre.itp']
2024-05-21 13:29:04,640 [MainThread  ] [INFO ]  Unzipping: 
2024-05-21 13:29:04,643 [MainThread  ] [INFO ]  3HTB_pdb2gmx_top.zip
2024-05-21 13:29:04,644 [MainThread  ] [INFO ]  To: 
2024-05-21 13:29:04,646 [MainThread  ] [INFO ]  2643cddf-ab09-4db9-a1e7-408a9c991886/p2g.top
2024-05-21 13:29:04,647 [MainThread  ] [INFO ]  2643cddf-ab09-4db9-a1e7-408a9c991886/posre.itp
2024-05-21 13:29:04,685 [MainThread  ] [INFO ]  Compressing topology to: 3HTB_JZ4_complex.top.zip
2024-05-21 13:29:04,687 [MainThread  ] [INFO ]  Ig

0

##Create solvent box

Define the unit cell for the protein-ligand complex to fill it with water molecules.
Truncated octahedron box is used for the unit cell. This box type is the one which best reflects the geometry of the solute/protein, in this case a globular protein, as it approximates a sphere. It is also convenient for the computational point of view, as it accumulates less water molecules at the corners, reducing the final number of water molecules in the system and making the simulation run faster.
A protein to box distance of 0.8 nm is used, and the protein is centered in the box.

Building Blocks used:

    Editconf from biobb_gromacs.gromacs.editconf


In [20]:
# Editconf: Create solvent box
# Import module
#from biobb_gromacs.gromacs.editconf import editconf

# Create prop dict and inputs/outputs
output_editconf_gro = pdbCode+'_'+ligandCode+'_complex_editconf.gro'

prop = {
    'box_type': 'octahedron',
    'distance_to_molecule': 0.8
}

# Create and launch bb
editconf(input_gro_path=complexFile_H,
         output_gro_path=output_editconf_gro,
         properties=prop)



2024-05-21 13:29:11,941 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.editconf Version: 4.1.0
2024-05-21 13:29:11,945 [MainThread  ] [INFO ]  Copy: 3HTB_JZ4_H.pdb to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/39f1b6d2-27b7-4861-b7d9-fa02ae2cd97b
2024-05-21 13:29:11,948 [MainThread  ] [INFO ]  Distance of the box to molecule:   0.80
2024-05-21 13:29:11,950 [MainThread  ] [INFO ]  Centering molecule in the box.
2024-05-21 13:29:11,951 [MainThread  ] [INFO ]  Box type: octahedron
2024-05-21 13:29:11,953 [MainThread  ] [INFO ]  GROMACS Editconf 20222 version detected
2024-05-21 13:29:11,954 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright editconf -f /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/39f1b6d2-27b7-4861-b7d9-fa02ae2cd97b/3HTB_JZ4_H.pdb -o /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/39f1b6d2-27b7-4861-b7d9-fa02ae2cd97b/3HTB_JZ4_complex_editco

0

###Fill the box with water molecules

Fill the unit cell for the protein-ligand complex with water molecules.
The solvent type used is the default Simple Point Charge water (SPC), a generic equilibrated 3-point solvent model.

Building Blocks used:

    Solvate from biobb_gromacs.gromacs.solvate


In [21]:
# Solvate: Fill the box with water molecules
#from biobb_gromacs.gromacs.solvate import solvate

# Create prop dict and inputs/outputs
output_solvate_gro = pdbCode+'_'+ligandCode+'_solvate.gro'
output_solvate_top_zip = pdbCode+'_'+ligandCode+'_solvate_top.zip'

# Create and launch bb
solvate(input_solute_gro_path=output_editconf_gro,
        output_gro_path=output_solvate_gro,
        input_top_zip_path=output_complex_top,
        output_top_zip_path=output_solvate_top_zip)

2024-05-21 13:29:19,270 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.solvate Version: 4.1.0
2024-05-21 13:29:19,275 [MainThread  ] [INFO ]  Copy: 3HTB_JZ4_complex_editconf.gro to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/fcf647ba-acb7-4ad8-9046-e8d5a5fcc924
2024-05-21 13:29:19,282 [MainThread  ] [INFO ]  Extracting: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/3HTB_JZ4_complex.top.zip
2024-05-21 13:29:19,283 [MainThread  ] [INFO ]  to:
2024-05-21 13:29:19,286 [MainThread  ] [INFO ]  ['b27d2684-1256-4728-ba18-b1555ef55215/JZ4_posres.itp', 'b27d2684-1256-4728-ba18-b1555ef55215/JZ4params.itp', 'b27d2684-1256-4728-ba18-b1555ef55215/ligand.top', 'b27d2684-1256-4728-ba18-b1555ef55215/posre.itp']
2024-05-21 13:29:19,288 [MainThread  ] [INFO ]  Unzipping: 
2024-05-21 13:29:19,290 [MainThread  ] [INFO ]  3HTB_JZ4_complex.top.zip
2024-05-21 13:29:19,291 [MainThread  ] [INFO ]  To: 
2024-05-21 13:29:

0


###Visualizing 3D structure

Visualizing the protein-ligand complex with the newly added solvent box using NGL
Note the octahedral box filled with water molecules surrounding the protein structure, which is centered right in the middle of the box.


In [22]:
#Show protein
view = nglview.show_structure_file(output_solvate_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='licorice', radius='.5', selection=ligandCode)
view.add_representation(repr_type='line', linewidth='1', selection='SOL', opacity='.3')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'

view.render_image()
view.download_image(filename='ngl7.png')

view

NGLWidget()

###Adding ions

Add ions to neutralize the protein-ligand complex and reach a desired ionic concentration.

    Step 1: Creating portable binary run file for ion generation
    Step 2: Adding ions to neutralize the system and reach a 0.05 molar ionic concentration


Building Blocks used:

    Grompp from biobb_gromacs.gromacs.grompp
    Genion from biobb_gromacs.gromacs.genion


Step 1: Creating portable binary run file for ion generation

In [23]:
# Grompp: Creating portable binary run file for ion generation
#from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
prop = {
    'mdp':{
        'nsteps':'5000'
    },
    'simulation_type':'minimization',
    'maxwarn': 1
}
output_gppion_tpr = pdbCode+'_'+ligandCode+'_complex_gppion.tpr'

# Create and launch bb
grompp(input_gro_path=output_solvate_gro,
       input_top_zip_path=output_solvate_top_zip,
       output_tpr_path=output_gppion_tpr,
       properties=prop)

2024-05-21 13:30:21,361 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.grompp Version: 4.1.0
2024-05-21 13:30:21,367 [MainThread  ] [INFO ]  Copy: 3HTB_JZ4_solvate.gro to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/452e67ba-5f5e-4064-abbf-e3755fed5ef5
2024-05-21 13:30:21,374 [MainThread  ] [INFO ]  Extracting: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/3HTB_JZ4_solvate_top.zip
2024-05-21 13:30:21,376 [MainThread  ] [INFO ]  to:
2024-05-21 13:30:21,378 [MainThread  ] [INFO ]  ['/content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/452e67ba-5f5e-4064-abbf-e3755fed5ef5/JZ4_posres.itp', '/content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/452e67ba-5f5e-4064-abbf-e3755fed5ef5/JZ4params.itp', '/content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/452e67ba-5f5e-4064-abbf-e3755fed5ef5/ligand.top', '/

0

### Step 2: Adding ions to neutralize the system and reach a 0.05 molar concentration

Replace solvent molecules with ions to neutralize the system and reaching a 0.05 molar ionic concentration

In [24]:
# Genion: Adding ions to reach a 0.05 molar concentration
#from biobb_gromacs.gromacs.genion import genion

# Create prop dict and inputs/outputs
prop={
    'neutral':True,
    'concentration':0.05
}
output_genion_gro = pdbCode+'_'+ligandCode+'_genion.gro'
output_genion_top_zip = pdbCode+'_'+ligandCode+'_genion_top.zip'

# Create and launch bb
genion(input_tpr_path=output_gppion_tpr,
       output_gro_path=output_genion_gro,
       input_top_zip_path=output_solvate_top_zip,
       output_top_zip_path=output_genion_top_zip,
       properties=prop)

2024-05-21 13:30:29,649 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.genion Version: 4.1.0
2024-05-21 13:30:29,654 [MainThread  ] [INFO ]  Copy: 3HTB_JZ4_complex_gppion.tpr to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/1b9965c3-1294-4af8-ac15-a2b4706df253
2024-05-21 13:30:29,657 [MainThread  ] [INFO ]  Copy: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/d1139a90-319c-4ac0-b9e4-65c8a348e887.stdin to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/1b9965c3-1294-4af8-ac15-a2b4706df253
2024-05-21 13:30:29,664 [MainThread  ] [INFO ]  Extracting: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/3HTB_JZ4_solvate_top.zip
2024-05-21 13:30:29,666 [MainThread  ] [INFO ]  to:
2024-05-21 13:30:29,668 [MainThread  ] [INFO ]  ['4fcac196-8b20-4192-ab9a-474df2c48bfe/JZ4_posres.itp', '4fcac196-8b20-4192-ab9a-474df2c48bfe/JZ4params.it

0


####Visualizing 3D structure

Visualizing the protein-ligand complex with the newly added ionic concentration using NGL


In [ ]:
#Show protein
view = nglview.show_structure_file(output_genion_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='licorice', radius='.5', selection=ligandCode)
view.add_representation(repr_type='ball+stick', selection='NA')
view.add_representation(repr_type='ball+stick', selection='CL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'

view.render_image()
view.download_image(filename='ngl8.png')

view

NGLWidget()

##Energetically minimize the system

Energetically minimize the protein-ligand complex till reaching a desired potential energy.

    Step 1: Creating portable binary run file for energy minimization
    Step 2: Energetically minimize the protein-ligand complex till reaching a force of 500 kJ/mol*nm.
    Step 3: Checking energy minimization results. Plotting energy by time during the minimization process.


Building Blocks used:

    Grompp from biobb_gromacs.gromacs.grompp
    Mdrun from biobb_gromacs.gromacs.mdrun
    GMXEnergy from biobb_analysis.gromacs.gmx_energy


###Step 1: Creating portable binary run file for energy minimization

Method used to run the energy minimization is a steepest descent, with a maximum force of 500 kJ/mol*nm^2, and a minimization step size of 1fs. The maximum number of steps to perform if the maximum force is not reached is 5,000 steps.

In [25]:
# Grompp: Creating portable binary run file for mdrun
#from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
prop = {
    'mdp':{
        'nsteps':'5000',
        'emstep': 0.01,
        'emtol':'500'
    },
    'simulation_type':'minimization'
}
output_gppmin_tpr = pdbCode+'_'+ligandCode+'_gppmin.tpr'

# Create and launch bb
grompp(input_gro_path=output_genion_gro,
       input_top_zip_path=output_genion_top_zip,
       output_tpr_path=output_gppmin_tpr,
       properties=prop)


2024-05-21 13:30:48,482 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.grompp Version: 4.1.0
2024-05-21 13:30:48,487 [MainThread  ] [INFO ]  Copy: 3HTB_JZ4_genion.gro to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/091ab2c4-ac27-4ae2-bf79-1dd90ed9ff5e
2024-05-21 13:30:48,493 [MainThread  ] [INFO ]  Extracting: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/3HTB_JZ4_genion_top.zip
2024-05-21 13:30:48,494 [MainThread  ] [INFO ]  to:
2024-05-21 13:30:48,496 [MainThread  ] [INFO ]  ['/content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/091ab2c4-ac27-4ae2-bf79-1dd90ed9ff5e/JZ4_posres.itp', '/content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/091ab2c4-ac27-4ae2-bf79-1dd90ed9ff5e/JZ4params.itp', '/content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/091ab2c4-ac27-4ae2-bf79-1dd90ed9ff5e/ligand.top', '/co

0

###Step 2: Running Energy Minimization

Running energy minimization using the tpr file generated in the previous step.

In [26]:
# Mdrun: Running minimization
#from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_min_trr = pdbCode+'_'+ligandCode+'_min.trr'
output_min_gro = pdbCode+'_'+ligandCode+'_min.gro'
output_min_edr = pdbCode+'_'+ligandCode+'_min.edr'
output_min_log = pdbCode+'_'+ligandCode+'_min.log'

# Create and launch bb
mdrun(input_tpr_path=output_gppmin_tpr,
      output_trr_path=output_min_trr,
      output_gro_path=output_min_gro,
      output_edr_path=output_min_edr,
      output_log_path=output_min_log)

2024-05-21 13:30:53,777 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.mdrun Version: 4.1.0
2024-05-21 13:30:53,782 [MainThread  ] [INFO ]  Copy: 3HTB_JZ4_gppmin.tpr to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/24628f3c-300d-49a1-8974-884119800eec
2024-05-21 13:30:53,785 [MainThread  ] [INFO ]  GROMACS Mdrun 20222 version detected
2024-05-21 13:30:53,787 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -o /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/24628f3c-300d-49a1-8974-884119800eec/3HTB_JZ4_min.trr -s /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/24628f3c-300d-49a1-8974-884119800eec/3HTB_JZ4_gppmin.tpr -c /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/24628f3c-300d-49a1-8974-884119800eec/3HTB_JZ4_min.gro -e /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/24628f3

0

###Step 3: Checking Energy Minimization results

Checking energy minimization results. Plotting potential energy by time during the minimization process.

In [27]:
# GMXEnergy: Getting system energy by time
#from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_min_ene_xvg = pdbCode+'_'+ligandCode+'_min_ene.xvg'
prop = {
    'terms':  ["Potential"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_min_edr,
          output_xvg_path=output_min_ene_xvg,
          properties=prop)

2024-05-21 13:32:35,589 [MainThread  ] [INFO ]  Executing biobb_analysis.gromacs.gmx_energy Version: 4.1.0
2024-05-21 13:32:35,593 [MainThread  ] [INFO ]  Copy: /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/3HTB_JZ4_min.edr to /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/6fde589c-15e8-4bf4-91f7-3810b0efdb30
2024-05-21 13:32:35,599 [MainThread  ] [INFO ]  gmx energy -f /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/6fde589c-15e8-4bf4-91f7-3810b0efdb30/3HTB_JZ4_min.edr -o /content/biobb_wf_protein-complex_md_setup/biobb_wf_protein-complex_md_setup/notebooks/6fde589c-15e8-4bf4-91f7-3810b0efdb30/3HTB_JZ4_min_ene.xvg -xvg none < 749d9600-cd06-4653-9daf-31d9217e41fa/instructions.in

2024-05-21 13:32:35,642 [MainThread  ] [INFO ]  Exit code 0

2024-05-21 13:32:35,645 [MainThread  ] [INFO ]  
Statistics over 1687 steps [ 0.0000 through 1686.0000 ps ], 1 data sets
All stati

0

In [ ]:
import plotly
from plotly.offline import iplot
import plotly.graph_objs as go


#Read data from file and filter energy values higher than 1000 kJ/mol
with open(output_min_ene_xvg,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file
            if not line.startswith(("#","@"))
            if float(line.split()[1]) < 1000
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = ({
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy kJ/mol")
                       )
})

plotly.offline.iplot(fig)
#fig.show(renderer="colab")

##Equilibrate the system (NVT)

Equilibrate the protein-ligand complex system in NVT ensemble (constant Number of particles, Volume and Temperature). To avoid temperature coupling problems, a new "system" group will be created including the protein + the ligand to be assigned to a single thermostatting group.

    Step 1: Creating an index file with a new group including the protein-ligand complex.
    Step 2: Creating portable binary run file for system equilibration
    Step 3: Equilibrate the protein-ligand complex with NVT ensemble.
    Step 4: Checking NVT Equilibration results. Plotting system temperature by time during the NVT equilibration process.


Building Blocks used:

    MakeNdx from biobb_gromacs.gromacs.make_ndx
    Grompp from biobb_gromacs.gromacs.grompp
    Mdrun from biobb_gromacs.gromacs.mdrun
    GMXEnergy from biobb_analysis.gromacs.gmx_energy


###Step 1: Creating an index file with a new group including the protein-ligand complex

In [ ]:
# MakeNdx: Creating index file with a new group (protein-ligand complex)
#from biobb_gromacs.gromacs.make_ndx import make_ndx

# Create prop dict and inputs/outputs
output_complex_ndx = pdbCode+'_'+ligandCode+'_index.ndx'
prop = {
    'selection': "\"Protein\"|\"Other\""
}

# Create and launch bb
make_ndx(input_structure_path=output_min_gro,
        output_ndx_path=output_complex_ndx,
        properties=prop)

###Step 2: Creating portable binary run file for system equilibration (NVT)

Note that for the purposes of temperature coupling, the protein-ligand complex (Protein_Other) is considered as a single entity.

In [ ]:
# Grompp: Creating portable binary run file for NVT System Equilibration
#from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppnvt_tpr = pdbCode+'_'+ligandCode+'gppnvt.tpr'
prop = {
    'mdp':{
        'nsteps':'5000',
        'tc-grps': 'Protein_Other Water_and_ions',
        'Define': '-DPOSRES -D' + posresifdef
    },
    'simulation_type':'nvt'
}

# Create and launch bb
grompp(input_gro_path=output_min_gro,
       input_top_zip_path=output_genion_top_zip,
       input_ndx_path=output_complex_ndx,
       output_tpr_path=output_gppnvt_tpr,
       properties=prop)

###Step 3: Running NVT equilibration

In [ ]:
# Mdrun: Running NVT System Equilibration
#from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_nvt_trr = pdbCode+'_'+ligandCode+'_nvt.trr'
output_nvt_gro = pdbCode+'_'+ligandCode+'_nvt.gro'
output_nvt_edr = pdbCode+'_'+ligandCode+'_nvt.edr'
output_nvt_log = pdbCode+'_'+ligandCode+'_nvt.log'
output_nvt_cpt = pdbCode+'_'+ligandCode+'_nvt.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppnvt_tpr,
      output_trr_path=output_nvt_trr,
      output_gro_path=output_nvt_gro,
      output_edr_path=output_nvt_edr,
      output_log_path=output_nvt_log,
      output_cpt_path=output_nvt_cpt)

###Step 4: Checking NVT Equilibration results

Checking NVT Equilibration results. Plotting system temperature by time during the NVT equilibration process.

In [ ]:
# GMXEnergy: Getting system temperature by time during NVT Equilibration
#from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_nvt_temp_xvg = pdbCode+'_'+ligandCode+'_nvt_temp.xvg'
prop = {
    'terms':  ["Temperature"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_nvt_edr,
          output_xvg_path=output_nvt_temp_xvg,
          properties=prop)

In [ ]:
import plotly
import plotly.graph_objs as go

# Read temperature data from file
with open(output_nvt_temp_xvg,'r') as temperature_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in temperature_file
            if not line.startswith(("#","@"))
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = ({
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Temperature during NVT Equilibration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Temperature (K)")
                       )
})

plotly.offline.iplot(fig)

##Equilibrate the system (NPT)

Equilibrate the protein-ligand complex system in NPT ensemble (constant Number of particles, Pressure and Temperature) .

    Step 1: Creating portable binary run file for system equilibration
    Step 2: Equilibrate the protein-ligand complex with NPT ensemble.
    Step 3: Checking NPT Equilibration results. Plotting system pressure and density by time during the NPT equilibration process.


###Step 1: Creating portable binary run file for system equilibration (NPT)

In [ ]:
# Grompp: Creating portable binary run file for (NPT) System Equilibration
#from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppnpt_tpr = pdbCode+'_'+ligandCode+'_gppnpt.tpr'
prop = {
    'mdp':{
        'type': 'npt',
        'nsteps':'5000',
        'tc-grps': 'Protein_Other Water_and_ions',
        'Define': '-DPOSRES -D' + posresifdef
    },
    'simulation_type':'npt'
}

# Create and launch bb
grompp(input_gro_path=output_nvt_gro,
       input_top_zip_path=output_genion_top_zip,
       input_ndx_path=output_complex_ndx,
       output_tpr_path=output_gppnpt_tpr,
       input_cpt_path=output_nvt_cpt,
       properties=prop)

###Step 2: Running NPT equilibration

In [ ]:
# Mdrun: Running NPT System Equilibration
#from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_npt_trr = pdbCode+'_'+ligandCode+'_npt.trr'
output_npt_gro = pdbCode+'_'+ligandCode+'_npt.gro'
output_npt_edr = pdbCode+'_'+ligandCode+'_npt.edr'
output_npt_log = pdbCode+'_'+ligandCode+'_npt.log'
output_npt_cpt = pdbCode+'_'+ligandCode+'_npt.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppnpt_tpr,
      output_trr_path=output_npt_trr,
      output_gro_path=output_npt_gro,
      output_edr_path=output_npt_edr,
      output_log_path=output_npt_log,
      output_cpt_path=output_npt_cpt)

###Step 3: Checking NPT Equilibration results

Checking NPT Equilibration results. Plotting system pressure and density by time during the NPT equilibration process.

In [ ]:
# GMXEnergy: Getting system pressure and density by time during NPT Equilibration
#from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_npt_pd_xvg = pdbCode+'_'+ligandCode+'_npt_PD.xvg'
prop = {
    'terms':  ["Pressure","Density"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_npt_edr,
          output_xvg_path=output_npt_pd_xvg,
          properties=prop)

In [ ]:
import plotly
from plotly import subplots
import plotly.graph_objs as go

# Read pressure and density data from file
with open(output_npt_pd_xvg,'r') as pd_file:
    x,y,z = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]),float(line.split()[2]))
            for line in pd_file
            if not line.startswith(("#","@"))
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

trace1 = go.Scatter(
    x=x,y=y
)
trace2 = go.Scatter(
    x=x,y=z
)

fig = subplots.make_subplots(rows=1, cols=2, print_grid=False)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout']['xaxis1'].update(title='Time (ps)')
fig['layout']['xaxis2'].update(title='Time (ps)')
fig['layout']['yaxis1'].update(title='Pressure (bar)')
fig['layout']['yaxis2'].update(title='Density (Kg*m^-3)')

fig['layout'].update(title='Pressure and Density during NPT Equilibration')
fig['layout'].update(showlegend=False)

plotly.offline.iplot(fig)


##Free Molecular Dynamics Simulation

Upon completion of the two equilibration phases (NVT and NPT), the system is now well-equilibrated at the desired temperature and pressure. The position restraints can now be released. The last step of the protein-ligand complex MD setup is a short, free MD simulation, to ensure the robustness of the system.

    Step 1: Creating portable binary run file to run a free MD simulation.
    Step 2: Run short MD simulation of the protein-ligand complex.
    Step 3: Checking results for the final step of the setup process, the free MD run. Plotting Root Mean Square deviation (RMSd) and Radius of Gyration (Rgyr) by time during the free MD run step.


Building Blocks used:

    Grompp from biobb_gromacs.gromacs.grompp
    Mdrun from biobb_gromacs.gromacs.mdrun
    GMXRms from biobb_analysis.gromacs.gmx_rms
    GMXRgyr from biobb_analysis.gromacs.gmx_rgyr


###Step 1: Creating portable binary run file to run a free MD simulation

In [ ]:
# Grompp: Creating portable binary run file for mdrun
#from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
prop = {
    'mdp':{
        #'nsteps':'500000' # 1 ns (500,000 steps x 2fs per step)
        #'nsteps':'5000' # 10 ps (5,000 steps x 2fs per step)
        'nsteps':'25000' # 50 ps (25,000 steps x 2fs per step)
    },
    'simulation_type':'free'
}
output_gppmd_tpr = pdbCode+'_'+ligandCode + '_gppmd.tpr'

# Create and launch bb
grompp(input_gro_path=output_npt_gro,
       input_top_zip_path=output_genion_top_zip,
       output_tpr_path=output_gppmd_tpr,
       input_cpt_path=output_npt_cpt,
       properties=prop)

###Step 2: Running short free MD simulation

In [ ]:
# Mdrun: Running free dynamics
#from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_md_trr = pdbCode+'_'+ligandCode+'_md.trr'
output_md_gro = pdbCode+'_'+ligandCode+'_md.gro'
output_md_edr = pdbCode+'_'+ligandCode+'_md.edr'
output_md_log = pdbCode+'_'+ligandCode+'_md.log'
output_md_cpt = pdbCode+'_'+ligandCode+'_md.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppmd_tpr,
      output_trr_path=output_md_trr,
      output_gro_path=output_md_gro,
      output_edr_path=output_md_edr,
      output_log_path=output_md_log,
      output_cpt_path=output_md_cpt)

###Step 3: Checking free MD simulation results

Checking results for the final step of the setup process, the free MD run. Plotting Root Mean Square deviation (RMSd) and Radius of Gyration (Rgyr) by time during the free MD run step. RMSd against the experimental structure (input structure of the pipeline) and against the minimized and equilibrated structure (output structure of the NPT equilibration step).

In [ ]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability
#         RMSd against minimized and equilibrated snapshot (backbone atoms)

#from biobb_analysis.gromacs.gmx_rms import gmx_rms

# Create prop dict and inputs/outputs
output_rms_first = pdbCode+'_'+ligandCode+'_rms_first.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
gmx_rms(input_structure_path=output_gppmd_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rms_first,
          properties=prop)

In [ ]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability
#         RMSd against experimental structure (backbone atoms)

#from biobb_analysis.gromacs.gmx_rms import gmx_rms

# Create prop dict and inputs/outputs
output_rms_exp = pdbCode+'_'+ligandCode+'_rms_exp.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
gmx_rms(input_structure_path=output_gppmin_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rms_exp,
          properties=prop)

In [ ]:
import plotly
import plotly.graph_objs as go

# Read RMS vs first snapshot data from file
with open(output_rms_first,'r') as rms_first_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_first_file
            if not line.startswith(("#","@"))
        ])
    )

# Read RMS vs experimental structure data from file
with open(output_rms_exp,'r') as rms_exp_file:
    x2,y2 = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_exp_file
            if not line.startswith(("#","@"))
        ])
    )

trace1 = go.Scatter(
    x = x,
    y = y,
    name = 'RMSd vs first'
)

trace2 = go.Scatter(
    x = x,
    y = y2,
    name = 'RMSd vs exp'
)

data = [trace1, trace2]

plotly.offline.init_notebook_mode(connected=True)

fig = ({
    "data": data,
    "layout": go.Layout(title="RMSd during free MD Simulation",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "RMSd (nm)")
                       )
})

plotly.offline.iplot(fig)


In [ ]:
# GMXRgyr: Computing Radius of Gyration to measure the protein compactness during the free MD simulation

#from biobb_analysis.gromacs.gmx_rgyr import gmx_rgyr

# Create prop dict and inputs/outputs
output_rgyr = pdbCode+'_'+ligandCode+'_rgyr.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
gmx_rgyr(input_structure_path=output_gppmin_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rgyr,
          properties=prop)


In [ ]:
import plotly
import plotly.graph_objs as go

# Read Rgyr data from file
with open(output_rgyr,'r') as rgyr_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rgyr_file
            if not line.startswith(("#","@"))
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = ({
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Radius of Gyration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Rgyr (nm)")
                       )
})

plotly.offline.iplot(fig)

##Post-processing and Visualizing resulting 3D trajectory

Post-processing and Visualizing the protein-ligand complex system MD setup resulting trajectory using NGL

    Step 1: Imaging the resulting trajectory, stripping out water molecules and ions and correcting periodicity issues.
    Step 2: Generating a dry structure, removing water molecules and ions from the final snapshot of the MD setup pipeline.
    Step 3: Visualizing the imaged trajectory using the dry structure as a topology.


Building Blocks used:

    GMXImage from biobb_analysis.gromacs.gmx_image
    GMXTrjConvStr from biobb_analysis.gromacs.gmx_trjconv_str


###Step 1: Imaging the resulting trajectory.

Stripping out water molecules and ions and correcting periodicity issues

In [ ]:
# GMXImage: "Imaging" the resulting trajectory
#           Removing water molecules and ions from the resulting structure
#from biobb_analysis.gromacs.gmx_image import gmx_image

# Create prop dict and inputs/outputs
output_imaged_traj = pdbCode+'_imaged_traj.trr'
prop = {
    'center_selection':  'Protein_Other',
    'output_selection': 'Protein_Other',
    'pbc' : 'mol',
    'center' : True
}

# Create and launch bb
gmx_image(input_traj_path=output_md_trr,
         input_top_path=output_gppmd_tpr,
         input_index_path=output_complex_ndx,
         output_traj_path=output_imaged_traj,
          properties=prop)

###Step 2: Generating the output dry structure.

Removing water molecules and ions from the resulting structure

In [ ]:
# GMXTrjConvStr: Converting and/or manipulating a structure
#                Removing water molecules and ions from the resulting structure
#                The "dry" structure will be used as a topology to visualize
#                the "imaged dry" trajectory generated in the previous step.
#from biobb_analysis.gromacs.gmx_trjconv_str import gmx_trjconv_str

# Create prop dict and inputs/outputs
output_dry_gro = pdbCode+'_md_dry.gro'
prop = {
    'selection':  'Protein_Other'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path=output_md_gro,
                input_top_path=output_gppmd_tpr,
                input_index_path=output_complex_ndx,
                output_str_path=output_dry_gro,
                properties=prop)

###Step 3: Visualizing the generated dehydrated trajectory.

Using the imaged trajectory (output of the Post-processing step 1) with the dry structure (output of the Post-processing step 2) as a topology.

In [ ]:
# Show trajectory
view = nglview.show_simpletraj(nglview.SimpletrajTrajectory(output_imaged_traj, output_dry_gro), gui=True)
view


NGLWidget(max_frame=5)

##Output files

Important Output files generated:

    3HTB_JZ4_md.gro: Final structure (snapshot) of the MD setup protocol.
    3HTB_JZ4_md.trr: Final trajectory of the MD setup protocol.
    3HTB_JZ4_md.cpt: Final checkpoint file, with information about the state of the simulation. It can be used to restart or continue a MD simulation.
    3HTB_JZ4_gppmd.tpr: Final tpr file, GROMACS portable binary run input file. This file contains the starting structure of the MD setup free MD simulation step, together with the molecular topology and all the simulation parameters. It can be used to extend the simulation.
    3HTB_JZ4_genion_top.zip: Final topology of the MD system. It is a compressed zip file including a topology file (.top) and a set of auxiliary include topology files (.itp).


Analysis (MD setup check) output files generated:

    3HTB_JZ4_rms_first.xvg: Root Mean Square deviation (RMSd) against minimized and equilibrated structure of the final free MD run step.
    3HTB_JZ4_rms_exp.xvg: Root Mean Square deviation (RMSd) against experimental structure of the final free MD run step.
    3HTB_JZ4_rgyr.xvg: Radius of Gyration of the final free MD run step of the setup pipeline.
